### DATA

需要分别读入face和pose的数据并对应起来
- 对于siblings_face 需要读入json  
- 对于siblings_pose 需要读入csv    
- 对于Label 需要读入csv


*特别注意对于face，需要去除置信度小于阈值的数据  
*要将video的label扩展到每个frame

In [11]:
import csv
import pandas as pd
import numpy as np

In [12]:
import os
import pandas as pd

# 读取标签文件
label = pd.read_csv('labelA/回应情况-表格 1.csv')

In [13]:
label.head()


,切片ID,回应情况,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,强,中,弱,没有回应（忽视）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15YS_20230317_01/VCAM_0000,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15YS_20230317_01/VCAM_0000_1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15YS_20230317_01/VCAM_0000_2,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15YS_20230317_01/VCAM_0000_3,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
label.iloc[1,0][-4:]

'0000'

导入face特征

In [52]:
# set confidence threshold
confidence_threshold = 0.5

# 导入face特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0
for i in range(1,len(label)):
    directory = label.iloc[i, 0]
    # cam = directory.split('/VCAM_')[-1]

    label_value = label.iloc[i, 1:5] # 取1-4位的标签
    feature_file = os.path.join('siblings_face', directory + '.csv')
    try:
        feature_data = pd.read_csv(feature_file)

        # 丢弃置信度小于0.8的数据
        feature_data = feature_data[feature_data['confidence'] >= confidence_threshold]
        
        # 每一frame加入
        features.append(feature_data.values) 

        # 重复frame标签
        for j in range(feature_data.shape[0]):
            labels.append(label_value.astype("int"))
        
        Dcount += 1
    except:
        print(f"error: {feature_file} not exist")
        Mcount +=1
        continue
    

print(f"Import {Dcount} files, missing {Mcount} files")

# 将特征和标签转换为numpy数组
X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"X:{len(X)}, y:{len(y)}")


error: siblings_face/15YS_20230317_01/VCAM_0000_3.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0000_37.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0001_29.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0001_30.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0001_37.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0002_53.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0002_76.csv not exist
error: siblings_face/15YS_20230317_01/VCAM_0003_89.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0004_126.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0006_18.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0006_81.csv not exist
error: siblings_face/16YS_20230317_01/VCAM_0006_103.csv not exist
Import 857 files, missing 12 files
X:19325, y:19325


In [ ]:
0.8 # 16000
0.7 # 17393
0.6 # 18295
0.5 # 19325

导入pose特征

In [40]:
import json

# 导入特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0

# for a single video
for i in range(1,len(label)):
    directory = label.iloc[i, 0] 
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    # label
    label_value = label.iloc[i, 1:5] # 取1-4位的标签

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    feature_data = []

    # for a single frame
    for file in json_files:
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            # data = list(data.values())
            rows = []
            for person in data['people']:
                row = []
                for keypoint in person['pose_keypoints_2d']:
                    if isinstance(keypoint, (int, float)):
                        row.append(keypoint)

                feature_data.append(np.array(row))

                # 重复frame标签
            
                # labels.append(label_value.astype("int"))
                labels.append(label_value.astype("int"))

                Dcount += 1
        except:
            print(f"error: {feature_file} not exist")
            Mcount +=1
            continue

    features.append(np.array(feature_data))
    

# 将特征和标签转换为numpy数组
X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"Import {Dcount} files, missing {Mcount} files")
print(f"X:{len(X)}, y:{len(y)}")

Import 52280 files, missing 0 files
X:52280, y:52280


In [45]:
X[0]

array([ 1.00000e+00,  0.00000e+00,  0.00000e+00,  6.20000e-01,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  1.41040e+03,  1.41190e+03,  1.41540e+03,
        1.41880e+03,  1.42020e+03,  1.41890e+03,  1.41510e+03,
        1.41170e+03,  1.40630e+03,  1.40810e+03,  1.41080e+03,
        1.41450e+03,  1.41830e+03,  1.42140e+03,  1.42370e+03,
        1.42110e+03,  1.41770e+03,  1.41410e+03,  1.41080e+03,
        1.40810e+03,  1.41380e+03,  1.41530e+03,  1.41690e+03,
        1.41750e+03,  1.41700e+03,  1.41550e+03,  1.41390e+03,
        1.41330e+03,  1.45630e+03,  1.45760e+03,  1.46140e+03,
        1.46530e+03,  1.46720e+03,  1.46580e+03,  1.46210e+03,
        1.45780e+03,  1.45270e+03,  1.45500e+03,  1.45820e+03,
        1.46240e+03,  1.46650e+03,  1.46980e+03,  1.47220e+03,
        1.47000e+03,  1.46680e+03,  1.46300e+03,  1.45910e+03,
        1.45540e+03,  1.46010e+03,  1.46180e+03,  1.463

In [10]:
y[0]

array([0, 0, 1, 0])

In [ ]:
import json

# 导入特征并对齐标签
features = []
labels = []
Mcount = 0
Dcount = 0

# for a single video
for i in range(1,len(label)):
    directory = label.iloc[i, 0] 
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    # label
    label_value = label.iloc[i, 1:5] # 取1-4位的标签

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    feature_data = []

    # for a single frame
    for file in json_files:
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            # data = list(data.values())
            rows = []
            for person in data['people']:
                row = []
                for keypoint in person['pose_keypoints_2d']:
                    if isinstance(keypoint, (int, float)):
                        row.append(keypoint)
                feature_data.append(np.array(row))

                # 重复frame标签
            
                # labels.append(label_value.astype("int"))
                labels.append(label_value.astype("int"))

                Dcount += 1
        except:
            print(f"error: {feature_file} not exist")
            Mcount +=1
            continue

    features.append(np.array(feature_data))
    

# 将特征和标签转换为numpy数组
X = np.concatenate(features, axis=0)
y = np.array(labels)

print(f"Import {Dcount} files, missing {Mcount} files")
print(f"X:{len(X)}, y:{len(y)}")

